In [3]:
!pip install pytorch-pretrained-bert
!pip install pytorch-nlp
!pip install Keras-Preprocessing
!pip install pandas
!pip install transformers

import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from torchnlp.datasets import imdb_dataset
from keras_preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter



D:\Applications\Anaconda\envs\fake_news_model\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Set pandas display options
pd.set_option('display.max_columns', None)

# Load the dataset
file_path = "D:\GITAM\8th sem\Project Phase - 2\IFND.csv"
df = pd.read_csv(file_path, encoding="ISO-8859-1")

# Encode the labels
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
df['Label'] = lb.fit_transform(df['Label'])

# Select relevant columns
df = df[['Statement', 'Label']]

# Check label distribution
print(Counter(df['Label'].values))

# Drop NaN values
df.dropna(inplace=True)

Counter({1: 37800, 0: 18914})


In [40]:
# Balance the dataset
df_fake = df[df['Label'] == 0]
df_nfake = df[df['Label'] == 1]
df_nfake_sampled = df_nfake.sample(n=len(df_fake))
df = pd.concat([df_nfake_sampled, df_fake], ignore_index=True)
df = df.sample(frac=1, random_state=24)
print(Counter(df['Label'].values))

Counter({1: 18914, 0: 18914})


In [16]:
# Split data into train and test sets
train_size = 35000  # Adjust this number as needed
test_size = 2000   # Adjust this number as needed

train_data = df.sample(n=train_size, random_state=42)
test_data = df.drop(train_data.index).sample(n=test_size, random_state=42)

# Prepare data for training
train_texts = train_data['Statement'].tolist()
train_labels = train_data['Label'].tolist()
test_texts = test_data['Statement'].tolist()
test_labels = test_data['Label'].tolist()

In [17]:
# Tokenize texts
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Convert encodings to tensors
train_tokens_tensor = torch.tensor(train_encodings['input_ids'])
train_masks_tensor = torch.tensor(train_encodings['attention_mask'])
train_y_tensor = torch.tensor(train_labels).float().unsqueeze(1)

test_tokens_tensor = torch.tensor(test_encodings['input_ids'])
test_masks_tensor = torch.tensor(test_encodings['attention_mask'])
test_y_tensor = torch.tensor(test_labels).float().unsqueeze(1)

D:\Applications\Anaconda\envs\fake_news_model\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anshu\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [19]:
# Define the model
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, tokens, masks=None):
        outputs = self.bert(tokens, attention_mask=masks)
        pooled_output = outputs.pooler_output
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [21]:
# Set up training parameters
BATCH_SIZE = 32
train_dataset = torch.utils.data.TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)

# Initialize the model
bert_clf = BertBinaryClassifier()

# Set optimizer
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)
EPOCHS = 5

In [22]:
# Training loop
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = batch_data
        probas = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_dataloader), train_loss / (step_num + 1)))

Epoch:  1
0/1094 loss: 0.6887074112892151 
Epoch:  1
1/1094 loss: 0.6754767894744873 
Epoch:  1
2/1094 loss: 0.6887283325195312 
Epoch:  1
3/1094 loss: 0.6848532557487488 
Epoch:  1
4/1094 loss: 0.6898488998413086 
Epoch:  1
5/1094 loss: 0.68452254931132 
Epoch:  1
6/1094 loss: 0.6810500025749207 
Epoch:  1
7/1094 loss: 0.6748959794640541 
Epoch:  1
8/1094 loss: 0.674409462345971 
Epoch:  1
9/1094 loss: 0.6727293908596039 
Epoch:  1
10/1094 loss: 0.6687774929133329 
Epoch:  1
11/1094 loss: 0.6675000240405401 
Epoch:  1
12/1094 loss: 0.6675883714969342 
Epoch:  1
13/1094 loss: 0.6663543879985809 
Epoch:  1
14/1094 loss: 0.6661113739013672 
Epoch:  1
15/1094 loss: 0.665711771696806 
Epoch:  1
16/1094 loss: 0.6620543669251835 
Epoch:  1
17/1094 loss: 0.659762418932385 
Epoch:  1
18/1094 loss: 0.6585917974773207 
Epoch:  1
19/1094 loss: 0.6588005900382996 
Epoch:  1
20/1094 loss: 0.6571016850925627 
Epoch:  1
21/1094 loss: 0.6561015221205625 
Epoch:  1
22/1094 loss: 0.6539671239645585 
Epo

In [23]:
# Set up test dataloader
test_dataset = torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)


# Evaluation
bert_clf.eval()
bert_predicted = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):
        token_ids, masks, labels = batch_data
        logits = bert_clf(token_ids, masks)
        bert_predicted += list(logits.cpu().numpy() > 0.5)

# Print classification report
print(classification_report(test_y_tensor.numpy(), bert_predicted))


              precision    recall  f1-score   support

         0.0       0.96      0.97      0.97      1013
         1.0       0.97      0.96      0.96       987

    accuracy                           0.97      2000
   macro avg       0.97      0.97      0.97      2000
weighted avg       0.97      0.97      0.97      2000



In [24]:
def predict_statement(statement):
    # Tokenize the statement
    encoded_statement = tokenizer(statement, truncation=True, padding=True, return_tensors='pt')
    
    # Convert tokenized statement into tensors
    input_ids = encoded_statement['input_ids']
    attention_mask = encoded_statement['attention_mask']
    
    # Pass the tensors through the model to get the prediction
    with torch.no_grad():
        logits = bert_clf(input_ids, attention_mask)
    
    # Convert logits to probabilities using sigmoid function
    probabilities = torch.sigmoid(logits)
    
    # Return the probabilities
    return probabilities.item()
statement = "COVID-19: Malaysian Couple Invites 10,000 Guests At Drive-Through Wedding"
prediction = predict_statement(statement)
print(prediction)

0.7301176190376282


In [65]:
def predict_statement(statement, threshold=0.6):
    encoded_statement = tokenizer(statement, truncation=True, padding=True, return_tensors='pt')
    input_ids = encoded_statement['input_ids']
    attention_mask = encoded_statement['attention_mask']
    with torch.no_grad():
        logits = bert_clf(input_ids, attention_mask)
    probabilities = torch.sigmoid(logits)
    prediction = 1 if probabilities.item() >= threshold else 0
    return prediction

statement = input()
prediction = predict_statement(statement)
if prediction == 1:
    print("True")
else:
    print("False")

 WHO critisizes aarogya setu app


False


In [33]:
# Save the trained model
# torch.save(bert_clf.state_dict(), 'Full_Bert_Model.pth')
torch.save(bert_clf.state_dict(), 'D:/GITAM/8th sem/Project Phase - 2/Bert Model/Full_Bert_Model.pth')

In [44]:
tokenizer.save_vocabulary("D:/GITAM/8th sem/Project Phase - 2/Bert Model/bert_tokenizer.vocab")

('D:/GITAM/8th sem/Project Phase - 2/Bert Model/bert_tokenizer.vocab',)